In [110]:
import pandas as pd
import numpy as np
import json
import os
import requests
import time
from dotenv import load_dotenv
from sklearn.preprocessing import MinMaxScaler
df=pd.read_csv('Sheet1.csv')
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

load_dotenv()

True

In [81]:
wallet_ids=list(df['wallet_id'].values)



In [93]:
api_key=os.getenv('api_key')
api_key2=os.getenv('api_key2')

In [94]:
compound_contracts={
    "cETH": "0x4ddc2d193948926d02f9b1fe9e1daa0718270ed5",
    "cDAI": "0x5d3a536e4d6dbd6114cc1ead35777bab948e3643",
    "cUSDC": "0x39aa39c021dfbae8fac545936693ac917d5e7563",
}

In [85]:
def fetch_compund_txns(wallet,chain_id='1'):  # chain_id='1'#universal identifier for Ethereum main network.
    url=f"https://api.covalenthq.com/v1/{chain_id}/address/{wallet}/transactions_v2/"
    params={
        'key':api_key
        
    }
    try:
        response=requests.get(url,params=params)
        if response.status_code==200:
            return response.json().get('data',{}).get('items',[])
        else:
            print(f'[ERROR] wallet {wallet}| Status"{response.status_code}')
            return []
    except Exception as e:
        print(f"[EXCEPTION] Wallet {wallet} | {str(e)}")
        return []
    
all_transactions=[]

for wallet in wallet_ids:
    txns=fetch_compund_txns(wallet)
    for txn in txns:
        to_address=txn.get('to_address','')#.lower()
        if to_address in [addr.lower() for addr in compound_contracts.values()]:
            all_transactions.append({
                "wallet": wallet,
                "hash": txn["tx_hash"],
                "value": txn["value"],
                "timestamp": txn["block_signed_at"],
                "contract": to_address,
                "gas_spent": txn["gas_spent"],
            })
        time.sleep(1)
        
    

[EXCEPTION] Wallet 0x623af911f493747c216ad389c7805a37019c662d | ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


In [92]:
len(all_transactions)

187

In [97]:
df = pd.DataFrame(all_transactions)

In [98]:
df.to_csv("compound_wallet_transactions.csv", index=False)
print("All data saved to compound_wallet_transactions.csv")

All data saved to compound_wallet_transactions.csv


In [100]:
df.head()

,wallet,hash,value,timestamp,contract,gas_spent
0,0x06b51c6882b27cb05e712185531c1f74996dd988,0x25b33b7a990646ab6c0f22d2c4e91fe563bbd9b02e36...,0,2020-10-16T00:16:57Z,0x39aa39c021dfbae8fac545936693ac917d5e7563,174552
1,0x0795732aacc448030ef374374eaae57d2965c16c,0x22d03ef2de5abcb5dd724279e59b050c995d6b6308ff...,0,2020-11-25T13:13:37Z,0x39aa39c021dfbae8fac545936693ac917d5e7563,177613
2,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9,0xe74e61815250aae8a2cab45a4ab4a98bcda6a3832591...,0,2020-12-12T06:15:09Z,0x39aa39c021dfbae8fac545936693ac917d5e7563,174525
3,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae,0x936f794a8eed05ee88765e3ddabb666473210fb18986...,0,2020-10-21T13:02:48Z,0x39aa39c021dfbae8fac545936693ac917d5e7563,174552
4,0x104ae61d8d487ad689969a17807ddc338b445416,0x44347e686635e8ae9efa4b2ba9703144ca6fd40921c7...,0,2020-12-16T05:19:37Z,0x39aa39c021dfbae8fac545936693ac917d5e7563,202745


In [101]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

features = df.groupby('wallet').agg(
    txn_count=('hash', 'count'),
    unique_contracts=('contract', pd.Series.nunique),
    total_value=('value', 'sum'),
    avg_gas_spent=('gas_spent', 'mean'),
    first_tx_time=('timestamp', 'min'),
    last_tx_time=('timestamp', 'max')
).reset_index()

# Compute derived features
features['active_days'] = (features['last_tx_time'] - features['first_tx_time']).dt.days + 1
features['frequency_per_day'] = features['txn_count'] / features['active_days']


In [102]:
features.to_csv("compound_wallet_features.csv", index=False)
print("Features saved to compound_wallet_features.csv")


Features saved to compound_wallet_features.csv


In [111]:
scaler = MinMaxScaler()
features_numeric = features[['txn_count', 'unique_contracts', 'total_value', 'avg_gas_spent', 'frequency_per_day']]
normalized = scaler.fit_transform(features_numeric)

In [112]:
features['score'] = normalized.mean(axis=1)



In [113]:
final_output = features[['wallet', 'score']].rename(columns={'wallet': 'wallet_id'})
final_output.to_csv("compound_wallet_scores.csv", index=False)
print("Deliverable saved to compound_wallet_scores.csv")

Deliverable saved to compound_wallet_scores.csv


In [117]:
final_output.head(3)

,wallet_id,score
0,0x06b51c6882b27cb05e712185531c1f74996dd988,0.127821
1,0x0795732aacc448030ef374374eaae57d2965c16c,0.129647
2,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9,0.127805
